Projet de web-scrapping du menu de Frichti pour savoir quels sont les combinaisons de plats les plus abordables et les plus onéreuses.

In [0]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import numpy as np

In [0]:
url = 'https://www.frichti.co/c/plats-cuisines-group/#entrees-1'
r=requests.get(url)
r.content
soup=BeautifulSoup(r.content,"html.parser")
a =soup.get_text()

Filtrage du texte brut et petit nettoyage pour enlever les informations correspondants aux formules et à des morceaux sans pertinence pour mon sujet.

In [0]:
r1 = re.findall(r",\"title\":\"(.*?)\,\"unit\":",a)

In [0]:
r1[:] = [x for x in r1 if not len(x)>375 
         if 'Cookie' not in x if 'B2B' not in x 
         if 'BUNDLE' not in x ]

In [0]:
r1=[row.split(',') for row in r1]

nettoyage du fichier et assemblage des noms de plats au sein d'un même élément.

In [0]:
for i in range(len(r1)):
  while r1[i][1].startswith('\"type')==False | r1[i][1].startswith('\"cat')==False:
    r1[i][0:2] = [''.join(r1[i][0:2])]
    if r1[i][1].startswith('\"type')==True | r1[i][1].startswith('\"cat')==True:
      break

In [0]:
for i in range(len(r1)):
  r1[i][:] = [j for j in r1[i] if ('type') not in j]

Construction d'une base de données des plats proposés avec leur catégorie et leur prix. Une part d'information a été mise de côté lorsque le prix des plats en question n'était pas renseigné dans les infos collectées. Un affinage du scrapping serait nécessaire pour compléter la base de données.

Etape intermédiaire: création de 3 listes de longueur identique à partir desquelles je vais créer une table de données

In [11]:
r1[0]

['Petit pain tradition"',
 '"category":"BOULANGERIE"',
 '"productId":2000600',
 '"price":0']

je souhaite garder le nom du plat (position 0), sa catégorie (position 1), et son prix (position 3). Le numéro d'identification du plat en position 2 ici ne m'intéresse pas.

In [0]:
plats=[r1[i][0] for i in range(len(r1))]
cat=[r1[i][1] for i in range(len(r1))]
price=[r1[i][3] for i in range(len(r1))]  

In [24]:
df = pd.DataFrame(list(zip(plats, cat,price)), columns =['PLATS', 'CATEGORY','PRICE']) 
df.CATEGORY = df.CATEGORY.str.strip('\"category\":\"')
df.PRICE = df.PRICE.str.strip('\"price\":\"')
df.PLATS=df.PLATS.str.strip('\"')

df.head()


,PLATS,CATEGORY,PRICE
0,Petit pain tradition,BOULANGERIE,0
1,Chili végétarien,PLAT,0
2,Chili végétarien,PLAT,0
3,Gaspacho acidulé de courgettes & sarriette,ENTREE,0
4,Petit pain tradition,BOULANGERIE,0


In [25]:
df1=df[df.PRICE!='0']
df1.PRICE = pd.to_numeric(df1.PRICE)
class_dict={'SNACK':'SNACK','JUS':'BOISSON','BIERE':'BOISSON','BOULANGERIE':'SNACK','CREMERIE':'DESSERT'}
classifications=['SNACK','JUS','BIERE','BOULANGERIE','CREMERIE']
def classification(a,b):
  df1.CATEGORY=df1.CATEGORY.apply(lambda x: np.where(a in x,b,x))
for i in classifications:
  classification(i,class_dict[i])

df1.PRICE=df1.PRICE.apply(lambda x : x/100)
df1.head()


/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,PLATS,CATEGORY,PRICE
8,Gaspacho acidulé de courgettes & sarriette,ENTREE,3.2
9,Carottes & feta AOP à la marocaine,ENTREE,3.5
10,Le choux chou,ENTREE,3.5
11,Carottes râpées au gingembre,ENTREE,3.6
12,Salade gourmande de pois chiches feta & herbes...,ENTREE,3.7


une simple étape esthétique pour avoir une vue d'ensemble par catégorie et par prix croissants.

In [26]:
df1.sort_values(['CATEGORY', 'PRICE'], ascending=[True, True], inplace=True)
df1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,PLATS,CATEGORY,PRICE
59,Evian,BOISSON,1.2
60,Coca-Cola,BOISSON,1.5
61,Coca-Cola Zéro,BOISSON,1.5
62,Badoit,BOISSON,1.5
66,Heineken 0.0 25cl,BOISSON,1.6
63,Eau de Coco Bio Cocomi,BOISSON,2.5
64,Jus d'orange Kookabarra (25cl),BOISSON,2.5
65,Pack de 6 Heineken 33cl,BOISSON,6.2
24,Yaourt Grec,DESSERT,2.0
25,Salade d'ananas zestissime,DESSERT,3.2


Etape finale avec construction des compositions les plus et les moins abordables, avec ou sans alcool, à partir de "sous-dataframes".

In [0]:
entree=df1[df1.CATEGORY=='ENTREE']
plat=df1[df1.CATEGORY=='PLAT']
dessert=df1[df1.CATEGORY=='DESSERT']
boisson = df1[df1.CATEGORY=='BOISSON']
snack = df1[df1.CATEGORY=='SNACK']
boisson['ALCOOL'] = np.where(boisson.PLATS.str.contains('Heineken', case=False), 0, 1)
noalcool=boisson[boisson.ALCOOL==1]

In [0]:
def selection_plat_min(x):
  return x.loc[x['PRICE'] == x.PRICE.min(), 'PLATS'].iloc[0]
def selection_plat_max(x):
  return x.loc[x['PRICE'] == x.PRICE.max(), 'PLATS'].iloc[0]

repas_min_dessert = [selection_plat_min(entree),selection_plat_min(plat),selection_plat_min(dessert),selection_plat_min(noalcool)]
repas_min_snack = [selection_plat_min(entree),selection_plat_min(plat),selection_plat_min(snack),selection_plat_min(noalcool)]

prix_min_snack=round(entree.PRICE.min()+plat.PRICE.min()+snack.PRICE.min()+noalcool.PRICE.min(),1)
prix_min_dessert=round(entree.PRICE.min()+plat.PRICE.min()+dessert.PRICE.min()+noalcool.PRICE.min(),1)

repas_sans_alcool_haut = [selection_plat_max(entree),selection_plat_max(plat),selection_plat_max(dessert),selection_plat_max(noalcool)]
repas_avec_alcool_haut = [selection_plat_max(entree),selection_plat_max(plat),selection_plat_max(dessert),selection_plat_max(boisson)]

prix_max_sans_a = round(entree.PRICE.max()+plat.PRICE.max()+snack.PRICE.max()+noalcool.PRICE.max(),1)
prix_max_avec_a = round(entree.PRICE.max()+plat.PRICE.max()+snack.PRICE.max()+boisson.PRICE.max(),1)
nl = ', \n'

repas_min_d= f"Le repas complet le plus abordable avec dessert proposé sur Frichti est composé de{nl}{nl.join(repas_min_dessert)} et il coûte {prix_min_dessert}€."
repas_min_s = f"Le repas complet le plus abordable avec snack proposé sur Frichti est composé de{nl}{nl.join(repas_min_snack)} et il coûte {prix_min_snack}€."
print(repas_min_s,'\n')

repas_sa_haut= f"Le repas complet le plus onéreux proposé sur Frichti est composé de{nl}{nl.join(repas_sans_alcool_haut)} et il coûte {prix_max_sans_a}€."
repas_aa_haut= f"Le repas complet le plus onéreux proposé sur Frichti est composé de{nl}{nl.join(repas_avec_alcool_haut)} et il coûte {prix_max_avec_a}€."
print(repas_aa_haut)


Reprise de tout le code précédent pour le transformer en programme qui sélectionne entre 2 choix : repas abordable ou onéreux.

In [0]:
def frichti():
  #STEP 1
  
  url = 'https://www.frichti.co/c/plats-cuisines-group/#entrees-1'
  r=requests.get(url)
  r.content
  soup=BeautifulSoup(r.content,"html.parser")
  a =soup.get_text()
  r1 = re.findall(r",\"title\":\"(.*?)\,\"unit\":",a)
  r1[:] = [x for x in r1 if not len(x)>375 if 'Cookie' not in x if 'B2B' not in x if 'BUNDLE' not in x ]
  r1=[row.split(',') for row in r1]
  for i in range(len(r1)):
    while r1[i][1].startswith('\"type')==False | r1[i][1].startswith('\"cat')==False:
      r1[i][0:2] = [''.join(r1[i][0:2])]
  for i in range(len(r1)):
    r1[i][:] = [j for j in r1[i] if ('type') not in j]
  
  #STEP 2
  plats=[r1[i][0] for i in range(len(r1))]
  cat=[r1[i][1] for i in range(len(r1))]
  price=[r1[i][3] for i in range(len(r1))] 

  df = pd.DataFrame(plats,columns =['PLATS'])
  df['CATEGORY']=cat
  df['PRICE']=price
  df.CATEGORY = df.CATEGORY.str.strip('\"category\":\"')
  df.PRICE = df.PRICE.str.strip('\"price\":\"')
  df.PLATS=df.PLATS.str.strip('\"')
  
  #STEP 3
  df1=df[df.PRICE!='0']
  df1.PRICE = pd.to_numeric(df1.PRICE)
  class_dict={'SNACK':'SNACK','JUS':'BOISSON','BIERE':'BOISSON','BOULANGERIE':'SNACK','CREMERIE':'DESSERT'}
  classifications=['SNACK','JUS','BIERE','BOULANGERIE','CREMERIE']
  def classification(a,b):
    df1.CATEGORY=df1.CATEGORY.apply(lambda x: np.where(a in x,b,x))
  for i in classifications:
    classification(i,class_dict[i])
  df1.PRICE=df1.PRICE.apply(lambda x : x/100)
  
  #STEP 4
  entree=df1[df1.CATEGORY=='ENTREE']
  plat=df1[df1.CATEGORY=='PLAT']
  dessert=df1[df1.CATEGORY=='DESSERT']
  boisson = df1[df1.CATEGORY=='BOISSON']
  boisson['ALCOOL'] = np.where(boisson.PLATS.str.contains('Heineken', case=False), 0, 1)
  noalcool=boisson[boisson.ALCOOL==1]
  
  #STEP 5
  def selection_plat_min(x):
    return x.loc[x['PRICE'] == x.PRICE.min(), 'PLATS'].iloc[0]
  def selection_plat_max(x):
    return x.loc[x['PRICE'] == x.PRICE.max(), 'PLATS'].iloc[0]
  nl = ', \n'
  
  #STEP 6
  message = 'Souhaitez-vous une suggestion pour \n une offre abordable (tapez 1) \n une offre plus onéreuse (tapez 2) :'
  error_mess = 'Seulement les chiffres 1 ou 2 sont des réponses valides. \n Merci de recommencer.'
  choice = input(message)
  while choice.isdigit() == False:
    print(error_mess)
    choice = input(message)
    if choice.isdigit() == True:
      break
  choice = int(choice)
  while (choice !=1) & (choice != 2) :
    print(error_mess)
    choice = int(input(message))
    if (choice ==1) | (choice == 2):
      break
      
  # OPTION 1
  if choice == 1:
    message2 = 'Souhaitez-vous une option \n avec un dessert (tapez 1) \n ou un snack plus léger (tapez 2) :'
    error_mess2 = 'Seulement les chiffres 1 ou 2 sont des réponses valides. \n Merci de recommencer.'
    choice2 = input(message2)
    # vérification que le message tapé ne puisse qu'être 1 ou 2
    while choice2.isdigit() == False:
      print(error_mess2)
      choice2 = input(message2)
      if choice2.isdigit() == True:
        break
    choice2 = int(choice2)
    while (choice2 !=1) & (choice2 != 2) :
      print(error_mess2)
      choice2 = int(input(message2))
      if (choice2 ==1) | (choice2 == 2):
        break
    # mise en forme des résultats à afficher
    repas_min_dessert = [selection_plat_min(entree),selection_plat_min(plat),selection_plat_min(dessert),selection_plat_min(noalcool)]
    repas_min_snack = [selection_plat_min(entree),selection_plat_min(plat),selection_plat_min(snack),selection_plat_min(noalcool)]
    prix_min_snack=round(entree.PRICE.min()+plat.PRICE.min()+snack.PRICE.min()+noalcool.PRICE.min(),1)
    prix_min_dessert=round(entree.PRICE.min()+plat.PRICE.min()+dessert.PRICE.min()+noalcool.PRICE.min(),1)
    repas_min_d= f"Le repas complet le plus abordable avec dessert proposé sur Frichti est composé de{nl}{nl.join(repas_min_dessert)} et il coûte {prix_min_dessert}€."
    repas_min_s = f"Le repas complet le plus abordable avec snack proposé sur Frichti est composé de{nl}{nl.join(repas_min_snack)} et il coûte {prix_min_snack}€."

    if choice2 == 1:
      print(repas_min_d,'\n')  
    if choice2 == 2:
      print(repas_min_s, '\n')  
  
  # OPTION 2
  if choice == 2:
    message3 = 'Souhaitez-vous une option \n avec alcool (tapez 1) \n ou sans (tapez 2) :'
    error_mess3 = 'Seulement les chiffres 1 ou 2 sont des réponses valides. \n Merci de recommencer.'
    choice3 = input(message3)
    # vérification que le message tapé ne puisse qu'être 1 ou 2
    while choice3.isdigit() == False:
      print(error_mess3)
      choice3 = input(message3)
      if choice3.isdigit() == True:
        break
    choice3 = int(choice3)
    while (choice3 !=1) & (choice3 != 2) :
      print(error_mess3)
      choice3 = int(input(message3))
      if (choice3 ==1) | (choice3 == 2):
        break
        
    repas_sans_alcool_haut = [selection_plat_max(entree),selection_plat_max(plat),selection_plat_max(dessert),selection_plat_max(noalcool)]
    repas_avec_alcool_haut = [selection_plat_max(entree),selection_plat_max(plat),selection_plat_max(dessert),selection_plat_max(boisson)]
    prix_max_sans_a = round(entree.PRICE.max()+plat.PRICE.max()+snack.PRICE.max()+noalcool.PRICE.max(),1)
    prix_max_avec_a = round(entree.PRICE.max()+plat.PRICE.max()+snack.PRICE.max()+boisson.PRICE.max(),1)
    repas_aa_haut= f"Le repas complet le plus onéreux avec alcool proposé sur Frichti est composé de{nl}{nl.join(repas_avec_alcool_haut)} et il coûte {prix_max_avec_a}€."
    repas_sa_haut= f"Le repas complet le plus onéreux sans alcool proposé sur Frichti est composé de{nl}{nl.join(repas_sans_alcool_haut)} et il coûte {prix_max_sans_a}€."
    
    if choice3 == 1:
      print(repas_aa_haut,'\n')  
    if choice3 == 2:
      print(repas_sa_haut) 
    
    

essai réussi avec le programme de sélection, vous pouvez le tester à votre tour : 

In [0]:
frichti()